# 1 - Merge de datasets

O objetivo desta etapa é unificar as bases, cruzando os valores com seus respectivos ID_CLIENTE.

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

## 1.1 - Base cadastral

### Sobre:
Arquivo: base_cadastral.csv

Base contendo informações cadastrais dos clientes. Cada cliente deve ter apenas uma data de cadastro e seus dados não mudam ao longo do tempo.
Variáveis:

* ID_CLIENTE: Identificador único do cliente.
* DATA_CADASTRO: Data da realização do cadastro no sistema.
* DDD: Número do DDD do telefone do cliente.
* FLAG_PF: Indica se o cliente é uma pessoa física (‘X’) ou jurídica (‘NaN’).
* SEGMENTO_INDUSTRIAL: Indica a qual segmento da indústria pertence o
cliente.
* DOMINIO_EMAIL: Indica o domínio(ou provedor) do email utilizado para o
cadastro.
* PORTE: Indica o porte (tamanho) da empresa.
* CEP_2_DIG: Indica os dois primeiros números do CEP do endereço
cadastrado.

In [ ]:
cadastro = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Datarisk/data/base_cadastral.csv')

In [ ]:
cadastro.shape

(1336, 9)

In [ ]:
cadastro.head()

,Unnamed: 0,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
0,0,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
1,1,8274986328479596038,2017-01-25,31,NaN,Comércio,YAHOO,MEDIO,77
2,2,345447888460137901,2000-08-15,75,NaN,Serviços,HOTMAIL,PEQUENO,48
3,3,1003144834589372198,2017-08-06,49,NaN,Serviços,OUTLOOK,PEQUENO,89
4,4,324916756972236008,2011-02-14,88,NaN,Serviços,GMAIL,GRANDE,62


Temos 1336 linhas e 9 colunas. Uma delas, a 'Unnamed: 0' será droppada após o merge.

## 1.2 Base de informações
### Sobre
Arquivo: base_info.csv

Base com informações adicionais dos clientes. Essa base é atualizada mensalmente, então cada cliente aparecerá apenas uma vez em cada mês de referência. Ou seja, o identificador único da base consiste na combinação do ID_CLIENTE e da SAFRA_REF.

Variáveis:
* ID_CLIENTE: Identificador único do cliente.
* SAFRA_REF: Mês de referência da amostra.
* RENDA_MES_ANTERIOR: Renda ou faturamento declarado pelo cliente no
fim do mês anterior.
* NO_FUNCIONARIOS: Número de funcionários reportado pelo cliente no fim
do mês anterior.


In [ ]:
info = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Datarisk/data/base_info.csv')

In [ ]:
info

,Unnamed: 0,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS
0,261993,1661240395903230676,2018-09,16913.0,92.0
1,390852,8274986328479596038,2018-09,106430.0,141.0
2,523225,345447888460137901,2018-09,707439.0,99.0
3,216838,1003144834589372198,2018-09,239659.0,96.0
4,214365,324916756972236008,2018-09,203123.0,103.0
...,...,...,...,...,...
24396,457392,705648002974742140,2021-12,278663.0,105.0
24397,112970,4993499380140734678,2021-12,156968.0,140.0
24398,222493,4614484019183480654,2021-12,292698.0,121.0
24399,481990,1299146298565441811,2021-12,106180.0,121.0


### Vamos agrupar os valores do dataframe usando média e mode

In [ ]:
# Função para calcular o mode
def mode_func(x):
    return pd.Series.mode(x)[0]
    
# Agrupando por ID_CLIENTE e criando um novo dataframe chamado info_new
info_new = info.groupby('ID_CLIENTE').agg({
    'RENDA_MES_ANTERIOR': 'mean',
    'NO_FUNCIONARIOS': 'mean',
    'SAFRA_REF': mode_func
})

In [ ]:
info_new

,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,SAFRA_REF
ID_CLIENTE,,,
8784237149961904,258803.825000,113.050000,2018-09
15070048310374146,407029.333333,78.000000,2021-01
18719614953021044,253016.555556,92.444444,2018-12
39547025441582855,317241.257143,105.228571,2018-09
43266641219008038,286908.000000,87.000000,2020-03
...,...,...,...
9161263096062288704,224236.785714,105.000000,2018-12
9175443729335908760,201655.857143,131.714286,2020-10
9184785003280841216,276220.756757,106.162162,2018-09


In [ ]:
# Merge nos datasets de cadastro e info_new para fazer merge na base de pagamentos em breve
cadastro_info = pd.merge(cadastro, info_new, on='ID_CLIENTE')

In [ ]:
cadastro_info

,Unnamed: 0,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,SAFRA_REF
0,0,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,290074.138889,99.361111,2018-09
1,1,8274986328479596038,2017-01-25,31,NaN,Comércio,YAHOO,MEDIO,77,172686.545455,144.636364,2018-09
2,2,345447888460137901,2000-08-15,75,NaN,Serviços,HOTMAIL,PEQUENO,48,317198.666667,110.761905,2018-09
3,3,1003144834589372198,2017-08-06,49,NaN,Serviços,OUTLOOK,PEQUENO,89,291218.974359,110.615385,2018-09
4,4,324916756972236008,2011-02-14,88,NaN,Serviços,GMAIL,GRANDE,62,303443.125000,111.700000,2018-09
...,...,...,...,...,...,...,...,...,...,...,...,...
1331,1331,957773253650890560,2021-07-05,NaN,NaN,Comércio,GMAIL,MEDIO,20,433808.000000,121.000000,2021-11
1332,1332,6094038865287329652,2021-07-05,NaN,NaN,Serviços,GMAIL,GRANDE,48,532236.000000,136.000000,2021-11
1333,1333,6893616200284527465,NaN,NaN,NaN,Comércio,HOTMAIL,MEDIO,na,108073.000000,129.000000,2021-12
1334,1334,7222128416864635362,NaN,NaN,NaN,Serviços,GMAIL,MEDIO,na,69394.000000,119.000000,2021-12


## 1.3Base de pagamentos
### Sobre

Arquivos:
* base_pagamentos (desenvolvimento do modelo)
* base_pagamentos_teste.csv (validação interna)

Base com informações sobre transações (empréstimos) passados. Cada cliente
pode ter uma ou mais transações no mesmo período de tempo.
* ID_CLIENTE: Identificador único do cliente.
* SAFRA_REF: Mês de referência da amostra.
* DATA_EMISSAO_DOCUMENTO: Data da emissão da nota de crédito.
* DATA_VENCIMENTO: Data limite para pagamento do empréstimo.
* VALOR_A_PAGAR: Valor da nota de crédito.
* TAXA: Taxa de juros cobrada no empréstimo.
* DATA_PAGAMENTO: Data em que o cliente realizou o pagamento da nota
(disponível apenas na base de desenvolvimento).


In [ ]:
pagtos = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Datarisk/data/base_pagamentos.csv')

In [ ]:
pagtos.shape

(89689, 8)

In [ ]:
pagtos.head()

,Unnamed: 0,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA
0,0,1661240395903230676,2018-09,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99
1,5,1661240395903230676,2018-09,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99
2,10,1661240395903230676,2018-09,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99
3,15,1661240395903230676,2018-09,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99
4,20,1661240395903230676,2018-09,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99


## 1.4 Criação da base de treino

In [ ]:
train = pd.merge(pagtos, cadastro_info, on='ID_CLIENTE')

In [ ]:
train.shape

(89689, 19)

In [ ]:
train

,Unnamed: 0_x,ID_CLIENTE,SAFRA_REF_x,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,Unnamed: 0_y,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,SAFRA_REF_y
0,0,1661240395903230676,2018-09,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99,0,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,290074.138889,99.361111,2018-09
1,5,1661240395903230676,2018-09,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99,0,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,290074.138889,99.361111,2018-09
2,10,1661240395903230676,2018-09,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99,0,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,290074.138889,99.361111,2018-09
3,15,1661240395903230676,2018-09,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99,0,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,290074.138889,99.361111,2018-09
4,20,1661240395903230676,2018-09,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99,0,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65,290074.138889,99.361111,2018-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89684,696245,957773253650890560,2021-11,2021-11-14,2021-12-06,2021-12-06,266.08,5.99,1331,2021-07-05,NaN,NaN,Comércio,GMAIL,MEDIO,20,433808.000000,121.000000,2021-11
89685,696249,6094038865287329652,2021-11,2021-11-14,2021-12-06,2021-12-06,301.49,8.99,1332,2021-07-05,NaN,NaN,Serviços,GMAIL,GRANDE,48,532236.000000,136.000000,2021-11
89686,696355,6893616200284527465,2021-12,2021-11-16,2021-12-08,2021-12-08,48520.00,5.99,1333,NaN,NaN,NaN,Comércio,HOTMAIL,MEDIO,na,108073.000000,129.000000,2021-12
89687,696374,7222128416864635362,2021-12,2021-11-16,2021-12-08,2021-12-08,71332.50,8.99,1334,NaN,NaN,NaN,Serviços,GMAIL,MEDIO,na,69394.000000,119.000000,2021-12


In [ ]:
train = train.drop(columns={'Unnamed: 0_x','Unnamed: 0_y'})

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89689 entries, 0 to 89688
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_CLIENTE              89689 non-null  int64  
 1   SAFRA_REF_x             89689 non-null  object 
 2   DATA_EMISSAO_DOCUMENTO  89689 non-null  object 
 3   DATA_PAGAMENTO          89689 non-null  object 
 4   DATA_VENCIMENTO         89689 non-null  object 
 5   VALOR_A_PAGAR           89689 non-null  float64
 6   TAXA                    89689 non-null  float64
 7   DATA_CADASTRO           89651 non-null  object 
 8   DDD                     80898 non-null  object 
 9   FLAG_PF                 266 non-null    object 
 10  SEGMENTO_INDUSTRIAL     88056 non-null  object 
 11  DOMINIO_EMAIL           88621 non-null  object 
 12  PORTE                   86834 non-null  object 
 13  CEP_2_DIG               89689 non-null  object 
 14  RENDA_MES_ANTERIOR      89689 non-null

### Realizando download do CSV

In [ ]:
train.to_csv('train.csv', index=False)